In [12]:
# Célula 1: 1. Bibliotecas
import pandas as pd
from icalendar import Calendar, Event # Importa Event também, embora não seja usado para escrita aqui
from datetime import datetime, date, time # Importa time para combine
import gspread
import os
from IPython.display import display # Para melhor visualização no Jupyter

#print("Bibliotecas importadas.")

In [13]:
# Célula 2: 2. Inputs (Configuração)

# Caminho para o arquivo .ics de entrada
#ical_file_path = 'calendars/c108_merged_booking_airbnb_google.ics' # Exemplo

# Chave (ID) da Planilha Google de destino
sheet_key = '1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY' # !!! SUBSTITUA PELA SUA CHAVE REAL !!!

# Caminho para o arquivo JSON de credenciais da conta de serviço
credentials_path = 'credentials2.json'

print("Configurações definidas:")
#print(f"- Arquivo iCal: {ical_file_path}")
print(f"- Chave Planilha: {sheet_key}")
print(f"- Credenciais: {credentials_path}")

Configurações definidas:
- Chave Planilha: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY
- Credenciais: credentials2.json


In [14]:
import pandas as pd
import os # Para verificar se o arquivo existe
from datetime import datetime # <-- ADICIONADO para buscar a hora atual

def get_reservas_inconsistentes(csv_file_path):
    """
    Lê um arquivo CSV de inconsistências, filtra para 'Origem' == 'OTA' e
    formata as linhas selecionadas em uma lista de listas com formato específico,
    incluindo um timestamp da sincronização.

    Args:
        csv_file_path (str): Caminho para o arquivo CSV contendo as inconsistências.
                             O CSV deve ter colunas incluindo 'Origem',
                             'Inicio', 'Fim', 'Summary'. (Docstring corrigido)

    Returns:
        list: Uma lista de listas, onde cada sub-lista tem o formato
              [Inicio("DD/MM/YYYY"), Fim("DD/MM/YYYY"), "", "", 
               "**SINCRONIZAÇÃO EM ...**", Summary]. (Docstring corrigido)
              Retorna uma lista vazia se nada for encontrado ou se ocorrer um erro.
    """
    resultado_final = []

    # --- 1. CAPTURAR O TIMESTAMP DA SINCRONIZAÇÃO ---
    # Captura a hora exata AGORA, uma vez por execução
    agora = datetime.now()
    # Formata a data/hora para o padrão solicitado
    timestamp_str = agora.strftime("%d/%m/%Y %H:%M:%S")
    # Cria a string final da coluna E
    texto_sincronizacao = f"**SINCRONIZAÇÃO EM {timestamp_str}**"
    # -----------------------------------------------

    # --- 2. Ler o arquivo CSV ---
    print(f"\nLendo arquivo CSV: {csv_file_path}")
    if not os.path.exists(csv_file_path):
        print(f"ERRO: Arquivo CSV '{csv_file_path}' não encontrado.")
        return resultado_final # Retorna lista vazia

    try:
        df_inconsistencias = pd.read_csv(csv_file_path)

        if df_inconsistencias.empty:
            print(f"Aviso: O arquivo CSV '{csv_file_path}' está vazio.")
            return resultado_final # Retorna lista vazia

    except pd.errors.EmptyDataError:
        print(f"Aviso: O arquivo CSV '{csv_file_path}' está vazio.")
        return resultado_final # Retorna lista vazia
    except Exception as e_read:
        print(f"ERRO ao ler o arquivo CSV '{csv_file_path}': {e_read}")
        return resultado_final # Retorna lista vazia
    # --- Fim da Leitura ---


    # --- 3. Processar o DataFrame ---
    # Verifica se as colunas necessárias existem (usando as colunas do seu código)
    colunas_necessarias = ['Origem', 'Inicio', 'Fim', 'Summary']
    if not all(col in df_inconsistencias.columns for col in colunas_necessarias):
        print("ERRO: O arquivo CSV não contém todas as colunas necessárias:", colunas_necessarias)
        return resultado_final # Retorna lista vazia

    try:
        # Filtra as linhas (usando a lógica do seu código)
        df_filtrado = df_inconsistencias[df_inconsistencias['Origem'] == 'OTA'].copy()

        # Garante que as colunas de data sejam datetime
        df_filtrado['Inicio'] = pd.to_datetime(df_filtrado['Inicio'], errors='coerce')
        df_filtrado['Fim'] = pd.to_datetime(df_filtrado['Fim'], errors='coerce')

        # Itera sobre as linhas filtradas
        for index, row in df_filtrado.iterrows():
            if pd.notna(row['Inicio']) and pd.notna(row['Fim']):
                # Formata as datas para "DD/MM/YYYY"
                inicio_formatado = row['Inicio'].strftime('%d/%m/%Y')
                fim_formatado = row['Fim'].strftime('%d/%m/%Y')

                # Obtém o Summary
                summary = str(row['Summary']) if pd.notna(row['Summary']) else ""

                # --- 4. MUDANÇA APLICADA AQUI ---
                # --- MONTAGEM DA LINHA (18 Colunas) ---
                # Mapeamento Ajustado:
                # Coluna 5 (índice 4) 'Quem' -> texto_sincronizacao
                # Coluna 6 (índice 5) 'Origem' -> summary
                
                linha_formatada = [
                    inicio_formatado,       # 1. Início
                    fim_formatado,          # 2. Fim
                    "",                     # 3. Dias
                    "",                     # 4. Pessoas
                    texto_sincronizacao,    # 5. Quem (TIMESTAMP) 
                    summary,                # 6. Origem (SUMMARY) 
                    "",                     # 7. Recebido
                    "",                     # 8. Despesas
                    "",                     # 9. A receber
                    "",                     # 10. Total Lq
                    "",                     # 11. Total BT
                    "",                     # 12. Diária BT
                    "",                     # 13. Diária Lq
                    "",                     # 14. Com.
                    "",                     # 15. Diária BT PAX
                    "",                     # 16. Diária Lq PAX
                    "",          # 17. Status
                    timestamp_str     # 18. Data Reserva
                ]
                # -------------------------------
                #Início	Fim	Dias	Pessoas	Quem	Origem	Recebido	Despesas	A receber	Total Lq	Total BT	Diária BT	Diária Lq	Com.	Diária BT PAX	Diária Lq PAX	Status	Data Reserva
                resultado_final.append(linha_formatada)
            else:
                # Mensagem de aviso corrigida para usar 'Inicio'/'Fim'
                print(f"Aviso: Pulando linha {index} do CSV devido a data inválida em 'Inicio' ou 'Fim'.")

    except KeyError as e:
        print(f"ERRO: Coluna não encontrada durante o processamento: {e}")
    except Exception as e:
        print(f"ERRO inesperado durante o processamento: {e}")

    # Retorna a lista de listas
    return resultado_final

# --- Exemplo de Uso ---
# (O código de exemplo permanece o mesmo)

'''arquivo_csv_entrada = 'calendars/ap201_inconsistencias.csv'

# Chamando a nova função
lista_para_planilha = get_reservas_inconsistentes(arquivo_csv_entrada)

if lista_para_planilha:
     print("\n--- Lista Formatada Resultante (pronta para Google Sheet) ---")
     for linha in lista_para_planilha:
         print(linha)
else:
     print("\nNenhuma linha formatada foi gerada.")'''


'arquivo_csv_entrada = \'calendars/ap201_inconsistencias.csv\'\n\n# Chamando a nova função\nlista_para_planilha = get_reservas_inconsistentes(arquivo_csv_entrada)\n\nif lista_para_planilha:\n     print("\n--- Lista Formatada Resultante (pronta para Google Sheet) ---")\n     for linha in lista_para_planilha:\n         print(linha)\nelse:\n     print("\nNenhuma linha formatada foi gerada.")'

In [15]:
import gspread
import pandas as pd # Usado apenas para exemplo, não estritamente necessário na função
import os
# (Presume que você tem as credenciais configuradas e as bibliotecas google-auth instaladas)

def inserir_linha_google_sheet(linha_dados, sheet_key, worksheet_name, credentials_path='credentials2.json'):
    """
    Insere uma única linha de dados ao final de uma aba específica em uma Planilha Google.

    Args:
        linha_dados (list): Uma lista de valores representando a linha a ser inserida.
                            A ordem dos valores deve corresponder à ordem das colunas na planilha.
                            Os valores podem ser números, strings, etc.
        sheet_key (str): A chave (ID) da Planilha Google.
        worksheet_name (str): O nome da aba (worksheet) onde a linha será inserida.
                              A aba DEVE existir.
        credentials_path (str): Caminho para o arquivo JSON da conta de serviço.
                                (Padrão: 'credentials.json').

    Returns:
        bool: True se a linha foi inserida com sucesso, False caso contrário.
    """
    print(f"--- Iniciando inserção de linha no Google Sheet ---")
    print(f"  Planilha Key: {sheet_key}")
    print(f"  Aba         : {worksheet_name}")
    print(f"  Dados da Linha: {linha_dados}")

    # Verifica se o arquivo de credenciais existe
    if not os.path.exists(credentials_path):
        print(f"ERRO: Arquivo de credenciais '{credentials_path}' não encontrado.")
        return False

    # Verifica se linha_dados é uma lista
    if not isinstance(linha_dados, list):
        print("ERRO: O parâmetro 'linha_dados' deve ser uma lista.")
        return False

    try:
        # 1. Autenticar
        print("Autenticando com conta de serviço...")
        gc = gspread.service_account(filename=credentials_path)
        print("Autenticação OK.")

        # 2. Abrir a Planilha
        print(f"Abrindo planilha...")
        sh = gc.open_by_key(sheet_key)
        print("Planilha aberta.")

        # 3. Selecionar a Aba (DEVE EXISTIR)
        try:
            worksheet = sh.worksheet(worksheet_name)
            print(f"Aba '{worksheet_name}' encontrada.")
        except gspread.exceptions.WorksheetNotFound:
            print(f"ERRO: Aba '{worksheet_name}' não encontrada na planilha.")
            print("Esta função não cria a aba, ela deve existir para inserir a linha.")
            return False # Não podemos continuar sem a aba

        # 4. Inserir (Append) a Linha
        print(f"Inserindo linha na aba '{worksheet_name}'...")
        # append_row adiciona a lista como uma nova linha após a última linha com conteúdo
        # 'USER_ENTERED' tenta interpretar os dados como se tivessem sido digitados
        # 
        for l in linha_dados:
            worksheet.insert_row(l, index=5, value_input_option='USER_ENTERED')
            #worksheet.append_row(l, value_input_option='USER_ENTERED')
            worksheet.sort((1, 'asc'), range='A4:Z10000') # Ordena após inserção
        print(f"{len(linha_dados)} Linhas inseridas com sucesso! na aba '{worksheet_name}'.")
        return True

    # Tratamento de Erros
    except gspread.exceptions.APIError as e_api:
        print(f"ERRO de API do Google: {e_api}")
        print("Verifique permissões de escrita, quotas e se a API está habilitada.")
        return False
    except gspread.exceptions.SpreadsheetNotFound:
         print(f"ERRO: Planilha com key '{sheet_key}' não encontrada ou sem permissão.")
         return False
    except Exception as e:
        print(f"ERRO inesperado durante a inserção da linha: {e}")
        return False

In [16]:
# --- Exemplo de Uso ---

# # Chave da sua planilha
SHEET_KEY = '1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY'
# # Nome da aba onde inserir
#WORKSHEET_NAME = 'Cópia de AP-201' # Ou qualquer outra aba existente
# # Caminho para as credenciais
CREDS_FILE = 'credentials2.json'

# --- Mapa de Apartamentos para seus respectivos arquivos de inconsistências ---
mapa_de_apartamentos = {
    'SM-C108': 'calendars/c108_inconsistencias.csv',
    'SM-D014': 'calendars/d014_inconsistencias.csv',
    'CBL004': 'calendars/cbl004_inconsistencias.csv',
    'AP-101': 'calendars/ap101_inconsistencias.csv',
    'AP-201': 'calendars/ap201_inconsistencias.csv',
}

# --- Loop sobre o dicionário de apartamentos ---
for aba, inconsistencias_file in mapa_de_apartamentos.items():
    try:
        if not os.path.exists(inconsistencias_file):
            print(f"Aviso: Arquivo '{inconsistencias_file}' para a aba '{aba}' não encontrado. Pulando.")
            continue
        else:
            novas_reservas = get_reservas_inconsistentes(inconsistencias_file)
            sucesso_insercao = inserir_linha_google_sheet(
                novas_reservas,
                SHEET_KEY,
                aba,
                CREDS_FILE
           )
            os.remove(inconsistencias_file)  # Remove o arquivo após o processamento
            if sucesso_insercao:
                print(f"Inserção de linhas para a aba '{aba}' concluída com sucesso!")

    except FileNotFoundError:
        print(f"Aviso: Arquivo '{inconsistencias_file}' para a aba '{aba}' não encontrado. Pulando.")
        continue
    except Exception as e:
        print(f"Erro ao ler o arquivo iCal '{inconsistencias_file}': {e}. Pulando aba '{aba}'.")
        continue


Aviso: Arquivo 'calendars/c108_inconsistencias.csv' para a aba 'SM-C108' não encontrado. Pulando.
Aviso: Arquivo 'calendars/d014_inconsistencias.csv' para a aba 'SM-D014' não encontrado. Pulando.
Aviso: Arquivo 'calendars/cbl004_inconsistencias.csv' para a aba 'CBL004' não encontrado. Pulando.

Lendo arquivo CSV: calendars/ap101_inconsistencias.csv
--- Iniciando inserção de linha no Google Sheet ---
  Planilha Key: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY
  Aba         : AP-101
  Dados da Linha: [['14/12/2025', '15/12/2025', '', '', '**SINCRONIZAÇÃO EM 24/11/2025 16:25:46**', 'Airbnb', '', '', '', '', '', '', '', '', '', '', '', '24/11/2025 16:25:46']]
Autenticando com conta de serviço...
Autenticação OK.
Abrindo planilha...
Planilha aberta.
Aba 'AP-101' encontrada.
Inserindo linha na aba 'AP-101'...
1 Linhas inseridas com sucesso! na aba 'AP-101'.
Inserção de linhas para a aba 'AP-101' concluída com sucesso!
Aviso: Arquivo 'calendars/ap201_inconsistencias.csv' para a aba 'AP-201' 